In [1]:
import os
import gc
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from scipy.stats import rankdata
import textstat
from tqdm import tqdm
from typing import Dict
import scml
import mylib

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda
NVIDIA GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [4]:
score_map: Dict[str, float] = {}
df = pd.read_csv("input/ruddit/Ruddit.csv", engine="c", low_memory=False)
for t in df.itertuples():
    k = getattr(t, "post_id") + "_" + getattr(t, "comment_id")
    score_map[k] = getattr(t, "offensiveness_score")

In [5]:
%%time
df = pd.read_csv("input/ruddit/ruddit_with_text.csv", engine="c", low_memory=False)
blacklist = {"[deleted]", "[removed]"}
rows = []
for t in df.itertuples():
    text = getattr(t, "txt")
    s = text.strip().lower()
    if len(s)==0 or s in blacklist:
        continue
    k = getattr(t, "post_id") + "_" + getattr(t, "comment_id")
    rows.append({"bws": score_map[k], "text": text})
df = pd.DataFrame.from_records(rows)
df["bws"] = df["bws"].astype(np.float32) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5710 entries, 0 to 5709
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   bws     5710 non-null   float32
 1   text    5710 non-null   object 
dtypes: float32(1), object(1)
memory usage: 67.0+ KB
Wall time: 45.7 ms


In [6]:
scml.find_missing_values(df)

,Total,Percent,Type
bws,0,0.0,float32
text,0,0.0,object


# Severity rating label

In [7]:
# Force unique ranks
col = "label"
df[col] = rankdata(df["bws"], method='ordinal')
df[col] = df[col].astype(np.int32)

# Preprocess Text

In [8]:
def preprocess(row) -> str:
    return mylib.preprocess(row["text"])


col = "text"
df[col] = df.progress_apply(preprocess, axis=1)

100%|█████████████████████████████████████████| 5710/5710 [00:02<00:00, 2434.99it/s]


In [9]:
cols = ["bws", "text"]
df[cols].sample(40).sort_values("bws").head(40)

,bws,text
5241,-0.646,I promise not to annoy you anymore.
1849,-0.562,"Yes, it is one the highest paying jobs per hour. People need to pay bills, eat, etcetera."
3541,-0.542,Expect Ceeday to come and ask for this
4058,-0.417,A lighter? May I ask why?
3447,-0.396,"I was joking. The loop is subtle, but the numbers make it obvious."
2763,-0.354,"Never felt those personally, but those are three out of a thousand"
1972,-0.333,Way too soon. At 14. In november I am turning 27 and my grandfather is not as bald as i am.
4542,-0.333,it seems they did not even bother to read the article
3226,-0.333,"Back when CollegeHumor was decent. Also, with the current trend in making movies out of board games, I am surprised this is not a real thing. Yet."
3265,-0.292,"Di tempatku lineage2 sekitar 2,05gb. Awal download kemarin sih sekitar 1,8."


# Character level features

In [10]:
%%time
col = "length"
df[col] = df["text"].str.len()
df[col] = df[col].astype(np.int16)

Wall time: 4.03 ms


In [11]:
def digit_frac(row) -> float:
    return mylib.digit_frac(row["text"])


def letter_frac(row) -> float:
    return mylib.letter_frac(row["text"])


def space_frac(row) -> float:
    return mylib.space_frac(row["text"])


def punc_frac(row) -> float:
    return mylib.punc_frac(row["text"])


def upper_frac(row) -> float:
    return mylib.upper_frac(row["text"])

In [12]:
col = "digit_frac"
df[col] = df.progress_apply(digit_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 37986.88it/s]


In [13]:
col = "letter_frac"
df[col] = df.progress_apply(letter_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 38257.03it/s]


In [14]:
col = "space_frac"
df[col] = df.progress_apply(space_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 38971.83it/s]


In [15]:
col = "punc_frac"
df[col] = df.progress_apply(punc_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 37253.13it/s]


In [16]:
col = "upper_frac"
df[col] = df.progress_apply(upper_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 38834.83it/s]


# Textstat features

In [17]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row["text"])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row["text"])


def sentence_count(row) -> int:
    return textstat.sentence_count(row["text"])


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row["text"])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row["text"])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row["text"])


def smog_index(row) -> float:
    return textstat.smog_index(row["text"])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row["text"])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row["text"])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row["text"])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row["text"])

In [18]:
col = "flesch_reading_ease"
df[col] = df.progress_apply(flesch_reading_ease, axis=1)
df[col] = df[col].astype(np.float32)

100%|█████████████████████████████████████████| 5710/5710 [00:00<00:00, 5752.24it/s]


In [19]:
col = "flesch_kincaid_grade"
df[col] = df.progress_apply(flesch_kincaid_grade, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 12404.20it/s]


In [20]:
col = "syllable_count"
df[col] = df.progress_apply(syllable_count, axis=1)
df[col] = df[col].astype(np.int16)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 18968.68it/s]


In [21]:
col = "lexicon_count"
df[col] = df.progress_apply(lexicon_count, axis=1)
df[col] = df[col].astype(np.int16)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 70244.60it/s]


In [22]:
col = "sentence_count"
df[col] = df.progress_apply(sentence_count, axis=1)
df[col] = df[col].astype(np.int16)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 40792.98it/s]


In [23]:
col = "gunning_fog"
df[col] = df.progress_apply(gunning_fog, axis=1)
df[col] = df[col].astype(np.float32)

100%|█████████████████████████████████████████| 5710/5710 [00:00<00:00, 9989.67it/s]


In [24]:
col = "smog_index"
df[col] = df.progress_apply(smog_index, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 16324.35it/s]


In [25]:
col = "automated_readability_index"
df[col] = df.progress_apply(automated_readability_index, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 27368.39it/s]


In [26]:
col = "coleman_liau_index"
df[col] = df.progress_apply(coleman_liau_index, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 24217.12it/s]


In [27]:
col = "linsear_write_formula"
df[col] = df.progress_apply(linsear_write_formula, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 11915.70it/s]


In [28]:
col = "dale_chall_readability_score"
df[col] = df.progress_apply(dale_chall_readability_score, axis=1)
df[col] = df[col].astype(np.float32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 10275.83it/s]


# Detoxify labels

In [29]:
sentences = list(df["text"])
batch_size = 256
dtfy_fs = []

In [30]:
%%time
prefix = "dto_"
res = mylib.detoxify_labels(
    sentences,
    checkpoint="pretrained/unitaryai/detoxify/toxic_original-c1212f89.ckpt",
    device=device,
    batch_size=batch_size
)
for k, v in res.items():
    col = prefix + k
    df[col] = v
    df[col] = df[col].astype(np.float32)
    dtfy_fs.append(col)
gc.collect()

Wall time: 1min 25s


4

In [31]:
%%time
prefix = "dtu_"
res = mylib.detoxify_labels(
    sentences,
    checkpoint="pretrained/unitaryai/detoxify/toxic_debiased-c7548aa0.ckpt",
    device=device,
    batch_size=batch_size
)
for k, v in res.items():
    col = prefix + k
    df[col] = v
    df[col] = df[col].astype(np.float32)
    dtfy_fs.append(col)
gc.collect()

Wall time: 1min 20s


4

In [32]:
%%time
prefix = "dtm_"
res = mylib.detoxify_labels(
    sentences,
    checkpoint="pretrained/unitaryai/detoxify/multilingual_debiased-0b549669.ckpt",
    device=device,
    batch_size=batch_size
)
for k, v in res.items():
    col = prefix + k
    df[col] = v
    df[col] = df[col].astype(np.float32)
    dtfy_fs.append(col)
gc.collect()

Wall time: 1min 29s


0

In [33]:
print(dtfy_fs)

['dto_toxicity', 'dto_severe_toxicity', 'dto_obscene', 'dto_threat', 'dto_insult', 'dto_identity_attack', 'dtu_toxicity', 'dtu_severe_toxicity', 'dtu_obscene', 'dtu_identity_attack', 'dtu_insult', 'dtu_threat', 'dtu_sexual_explicit', 'dtm_toxicity', 'dtm_severe_toxicity', 'dtm_obscene', 'dtm_identity_attack', 'dtm_insult', 'dtm_threat', 'dtm_sexual_explicit']


# Embeddings

In [34]:
model = SentenceTransformer("pretrained/sentence-transformers/paraphrase-MiniLM-L6-v2", device=device)
model.max_seq_length = 128

In [35]:
%%time
em = model.encode(sentences=sentences, batch_size=1000, show_progress_bar=True, convert_to_numpy=True)
print(f"em.shape={em.shape}")

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

em.shape=(5710, 384)
Wall time: 6.4 s


In [36]:
%%time
em_size = em.shape[1]
em_cols = [f"zz{i:04d}" for i in range(em_size)]
df[em_cols] = em
df[em_cols] = df[em_cols].astype(np.float32)
del sentences

s:\dev\seahrh\kaggle-jigsaw-toxic-severity-rating\env\lib\site-packages\pandas\core\frame.py:3673: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


Wall time: 234 ms


# Review data

In [37]:
col = "worker"
df[col] = 0
df[col] = df[col].astype(np.int8)
cols = ["label", "bws", "worker"]
char_fs = ["length", "digit_frac", "letter_frac", "space_frac", "punc_frac", "upper_frac"]
textstat_fs = ["syllable_count", "lexicon_count", "sentence_count", "flesch_reading_ease", 
           "flesch_kincaid_grade", "gunning_fog", "smog_index", "automated_readability_index", 
           "coleman_liau_index", "linsear_write_formula", "dale_chall_readability_score"]
cols += char_fs + textstat_fs + dtfy_fs
df[cols].describe(percentiles=percentiles)

,label,bws,worker,length,digit_frac,letter_frac,space_frac,punc_frac,upper_frac,syllable_count,lexicon_count,sentence_count,flesch_reading_ease,flesch_kincaid_grade,gunning_fog,smog_index,automated_readability_index,coleman_liau_index,linsear_write_formula,dale_chall_readability_score,dto_toxicity,dto_severe_toxicity,dto_obscene,dto_threat,dto_insult,dto_identity_attack,dtu_toxicity,dtu_severe_toxicity,dtu_obscene,dtu_identity_attack,dtu_insult,dtu_threat,dtu_sexual_explicit,dtm_toxicity,dtm_severe_toxicity,dtm_obscene,dtm_identity_attack,dtm_insult,dtm_threat,dtm_sexual_explicit
count,5710.00000,5710.000000,5710.0,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5.710000e+03,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000
mean,2855.50000,-0.027706,0.0,197.564098,0.003542,0.788830,0.177722,0.029905,0.030517,50.067426,36.371278,2.319790,75.253105,6.685254,8.998004,3.062995,7.482505,6.631683,8.562484,8.299163,0.177856,0.013080,0.113673,0.008411,0.059683,0.011454,0.195873,4.698273e-03,0.113027,0.015234,0.081492,0.011010,0.040167,0.203075,0.005742,0.105158,0.013474,0.083563,0.014735,0.044618
std,1648.47935,0.334195,0.0,172.016744,0.012983,0.034318,0.021473,0.024070,0.049035,44.035705,31.194250,1.722369,19.903004,4.430519,4.645890,4.695458,5.593331,3.883116,5.906460,2.270676,0.326334,0.052992,0.282462,0.056317,0.175381,0.064708,0.341383,2.272458e-02,0.285412,0.074354,0.207780,0.069685,0.149433,0.345334,0.027226,0.272925,0.075226,0.214901,0.078828,0.155314
min,1.00000,-0.889000,0.0,15.000000,0.000000,0.400000,0.040541,0.000000,0.000000,3.000000,2.000000,1.000000,-48.980000,-2.500000,0.800000,0.000000,-8.700000,-10.160000,0.000000,0.200000,0.000506,0.000080,0.000141,0.000086,0.000164,0.000121,0.000286,9.529070e-07,0.000017,0.000052,0.000061,0.000012,0.000009,0.000178,0.000009,0.000063,0.000045,0.000095,0.000014,0.000012
1%,58.09000,-0.667000,0.0,24.000000,0.000000,0.672759,0.117647,0.000000,0.000000,6.000000,5.000000,1.000000,19.430301,-1.500000,2.000000,0.000000,-3.000000,-3.003700,1.500000,0.350000,0.000556,0.000087,0.000158,0.000096,0.000171,0.000134,0.000360,1.087282e-06,0.000021,0.000063,0.000092,0.000015,0.000011,0.000281,0.000013,0.000095,0.000066,0.000149,0.000021,0.000017
5%,286.45000,-0.521000,0.0,33.000000,0.000000,0.733333,0.141414,0.000000,0.005062,8.000000,6.000000,1.000000,41.869999,0.500000,2.400000,0.000000,-0.200000,0.320000,2.000000,5.620000,0.000614,0.000093,0.000165,0.000103,0.000175,0.000138,0.000410,1.200621e-06,0.000024,0.000070,0.000103,0.000017,0.000012,0.000357,0.000016,0.000115,0.000078,0.000177,0.000027,0.000020
10%,571.90000,-0.426000,0.0,42.000000,0.000000,0.750000,0.151515,0.010417,0.008333,10.000000,8.000000,1.000000,50.669998,1.700000,3.200000,0.000000,1.290000,1.820000,2.741667,6.240000,0.000668,0.000097,0.000170,0.000108,0.000177,0.000140,0.000454,1.271093e-06,0.000026,0.000076,0.000111,0.000019,0.000013,0.000417,0.000019,0.000129,0.000086,0.000200,0.000031,0.000022
20%,1142.80000,-0.312000,0.0,60.000000,0.000000,0.769841,0.161966,0.015385,0.011758,15.000000,11.000000,1.000000,59.980000,3.100000,4.870000,0.000000,3.200000,3.698000,4.000000,6.930000,0.000797,0.000103,0.000176,0.000115,0.000181,0.000145,0.000572,1.407266e-06,0.000031,0.000089,0.000133,0.000023,0.000016,0.000578,0.000023,0.000159,0.000099,0.000247,0.000037,0.000026
30%,1713.70000,-0.213000,0.0,82.000000,0.000000,0.779661,0.168627,0.018692,0.014599,20.000000,15.000000,1.000000,66.739998,4.400000,6.364000,0.000000,4.600000,4.930000,5.000000,7.380000,0.001006,0.000108,0.000183,0.000121,0.000188,0.000151,0.000850,1.645227e-06,0.000041,0.000113,0.000177,0.000030,0.000021,0.000857,0.000027,0.000201,0.000118,0.00

In [38]:
cols += em_cols
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5710 entries, 0 to 5709
Data columns (total 424 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    label                         5710 non-null   int32  
 1    bws                           5710 non-null   float32
 2    worker                        5710 non-null   int8   
 3    length                        5710 non-null   int16  
 4    digit_frac                    5710 non-null   float32
 5    letter_frac                   5710 non-null   float32
 6    space_frac                    5710 non-null   float32
 7    punc_frac                     5710 non-null   float32
 8    upper_frac                    5710 non-null   float32
 9    syllable_count                5710 non-null   int16  
 10   lexicon_count                 5710 non-null   int16  
 11   sentence_count                5710 non-null   int16  
 12   flesch_reading_ease           5710 non-null   

In [39]:
%%time
df[cols].to_parquet("output/ruddit.parquet", index=False)

Wall time: 223 ms
